In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
import cv2
import numpy as np

# Path to car images
car_images_path = r'C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test'

# Load car images
car_images = []
for filename in os.listdir(car_images_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other formats if needed
        image_path = os.path.join(car_images_path, filename)
        image = cv2.imread(image_path)
        if image is not None:
            car_images.append(image)

print(f"Loaded {len(car_images)} car images.")

Loaded 210 car images.


In [3]:
# Path to license plate images
license_plate_images_path = r'C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\Licplatesdetection_train\license_plates_detection_train'

# Load license plate images
license_plate_images = []
for filename in os.listdir(license_plate_images_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other formats if needed
        image_path = os.path.join(license_plate_images_path, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale for character recognition
        if image is not None:
            license_plate_images.append(image)

print(f"Loaded {len(license_plate_images)} license plate images.")

Loaded 900 license plate images.


In [4]:
def preprocess_car_image(image, target_size=(128, 128)):
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize to [0, 1]
    return image

preprocessed_car_images = [preprocess_car_image(img) for img in car_images]

In [5]:
def preprocess_license_plate(image, target_size=(64, 64)):
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize to [0, 1]
    return image

preprocessed_license_plates = [preprocess_license_plate(img) for img in license_plate_images]

In [12]:
import easyocr
import cv2
import os
import pandas as pd

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])

# Path to the folder containing car images
car_images_path = r'C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test'

# Output CSV file to save annotations
output_csv = 'automatic_annotations.csv'

# Write headers to the CSV file
with open(output_csv, 'w') as file:
    file.write("filename,xmin,ymin,xmax,ymax,text,confidence\n")

# Loop through each image in the folder
for filename in os.listdir(car_images_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(car_images_path, filename)
        print(f"Processing {image_path}...")

        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to load image at {image_path}")
            continue

        # Perform text detection
        results = reader.readtext(image)

        # Save detected text and bounding boxes to CSV
        with open(output_csv, 'a') as file:
            for result in results:
                bbox, text, confidence = result
                xmin, ymin = map(int, bbox[0])
                xmax, ymax = map(int, bbox[2])
                file.write(f"{filename},{xmin},{ymin},{xmax},{ymax},{text},{confidence}\n")

print("Automatic annotation process completed.")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Processing C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test\1000.jpg...
Processing C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test\1001.jpg...
Processing C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test\1002.jpg...
Processing C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test\1003.jpg...
Processing C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test\1004.jpg...
Processing C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test\1005.jpg...
Processing C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test\1006.jpg...
Processing C:\Users\user\Downloads\DATA SCIENTIST_ASSIG

In [14]:
import os
print(os.getcwd())

C:\Users\user


In [16]:
import os
import cv2
import numpy as np
import pandas as pd

# Load the annotations
annotations = pd.read_csv('automatic_annotations.csv')

# Path to the folder containing car images
car_images_path = r'C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test'

# Path to save cropped license plate images
output_folder = 'cropped_license_plates'
os.makedirs(output_folder, exist_ok=True)

# Preprocess and save cropped license plates
cropped_images = []
text_labels = []

for index, row in annotations.iterrows():
    filename = row['filename']
    xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
    text = row['text']

    # Load the image
    image_path = os.path.join(car_images_path, filename)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        continue

    # Get image dimensions
    height, width, _ = image.shape

    # Validate bounding box coordinates
    if xmin >= xmax or ymin >= ymax:
        print(f"Invalid bounding box for {filename}: ({xmin}, {ymin}, {xmax}, {ymax})")
        continue
    if xmin < 0 or ymin < 0 or xmax > width or ymax > height:
        print(f"Bounding box out of bounds for {filename}: ({xmin}, {ymin}, {xmax}, {ymax})")
        continue

    # Crop the license plate
    license_plate = image[ymin:ymax, xmin:xmax]

    # Check if the cropped image is valid
    if license_plate.size == 0:
        print(f"Empty license plate for {filename}: ({xmin}, {ymin}, {xmax}, {ymax})")
        continue

    # Resize and normalize the license plate
    license_plate = cv2.resize(license_plate, (128, 64))  # Adjust size as needed
    license_plate = license_plate / 255.0  # Normalize to [0, 1]

    # Save the cropped license plate
    output_path = os.path.join(output_folder, f"cropped_{filename}")
    cv2.imwrite(output_path, license_plate * 255)  # Save as 8-bit image

    # Append to lists
    cropped_images.append(license_plate)
    text_labels.append(text)

# Convert to numpy arrays
cropped_images = np.array(cropped_images)
text_labels = np.array(text_labels)

print(f"Preprocessed {len(cropped_images)} license plates.")

Invalid bounding box for 1066.jpg: (218, 449, 326, 449)
Bounding box out of bounds for 1092.jpg: (766, 7, 855, 45)
Invalid bounding box for 907.jpg: (185, 198, 312, 198)
Preprocessed 402 license plates.


In [17]:
import os
import cv2
import numpy as np
import pandas as pd

# Load the annotations
annotations = pd.read_csv('automatic_annotations.csv')

# Path to the folder containing car images
car_images_path = r'C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test'

# Path to save cropped license plate images
output_folder = 'cropped_license_plates'
os.makedirs(output_folder, exist_ok=True)

# Preprocess and save cropped license plates
cropped_images = []
text_labels = []

for index, row in annotations.iterrows():
    filename = row['filename']
    xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
    text = row['text']

    # Load the image
    image_path = os.path.join(car_images_path, filename)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        continue

    # Get image dimensions
    height, width, _ = image.shape

    # Validate bounding box coordinates
    if xmin >= xmax or ymin >= ymax:
        print(f"Invalid bounding box for {filename}: ({xmin}, {ymin}, {xmax}, {ymax})")
        continue
    if xmin < 0 or ymin < 0 or xmax > width or ymax > height:
        print(f"Bounding box out of bounds for {filename}: ({xmin}, {ymin}, {xmax}, {ymax})")
        continue

    # Crop the license plate
    license_plate = image[ymin:ymax, xmin:xmax]

    # Check if the cropped image is valid
    if license_plate.size == 0:
        print(f"Empty license plate for {filename}: ({xmin}, {ymin}, {xmax}, {ymax})")
        continue

    # Resize and normalize the license plate
    license_plate = cv2.resize(license_plate, (128, 64))  # Adjust size as needed
    license_plate = license_plate / 255.0  # Normalize to [0, 1]

    # Save the cropped license plate
    output_path = os.path.join(output_folder, f"cropped_{filename}")
    cv2.imwrite(output_path, license_plate * 255)  # Save as 8-bit image

    # Append to lists
    cropped_images.append(license_plate)
    text_labels.append(text)

# Convert to numpy arrays
cropped_images = np.array(cropped_images)
text_labels = np.array(text_labels)

print(f"Preprocessed {len(cropped_images)} license plates.")

Invalid bounding box for 1066.jpg: (218, 449, 326, 449)
Bounding box out of bounds for 1092.jpg: (766, 7, 855, 45)
Invalid bounding box for 907.jpg: (185, 198, 312, 198)
Preprocessed 402 license plates.


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode text labels into numerical labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(text_labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cropped_images, encoded_labels, test_size=0.2, random_state=42)

# Define the CNN model
def create_recognition_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the model
input_shape = (64, 128, 3)  # Adjust based on your image size
num_classes = len(label_encoder.classes_)
recognition_model = create_recognition_model(input_shape, num_classes)
recognition_model.summary()

# Train the model
recognition_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 126, 32)         │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 26880)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,440,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 375)                 │          48,375 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,508,535 (13.38 MB)

 Trainable params: 3,508,535 (13.38 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 196ms/step - accuracy: 0.0000e+00 - loss: 6.0797 - val_accuracy: 0.0000e+00 - val_loss: 5.9396
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 135ms/step - accuracy: 0.0161 - loss: 5.9162 - val_accuracy: 0.0000e+00 - val_loss: 5.9759
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 137ms/step - accuracy: 0.0278 - loss: 5.9089 - val_accuracy: 0.0000e+00 - val_loss: 5.9815
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.0171 - loss: 5.8763 - val_accuracy: 0.0000e+00 - val_loss: 6.0456
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.0069 - loss: 5.8376 - val_accuracy: 0.0000e+00 - val_loss: 6.2194
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.0225 - loss: 5.7384 - val_accuracy: 0.0000e+00 - val_loss: 6.5616
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.0314 - loss: 5.6154 - val_accuracy: 0.0000e+00 - val_loss: 6.9030
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.0568 -

In [19]:
# Evaluate the model
test_loss, test_accuracy = recognition_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Predict on a sample image
sample_image = X_test[0]
predicted_label = recognition_model.predict(np.expand_dims(sample_image, axis=0))
predicted_text = label_encoder.inverse_transform([np.argmax(predicted_label)])[0]
print(f"Predicted Text: {predicted_text}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0280 - loss: 8.0994
Test Accuracy: 2.47%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
Predicted Text: 3973


In [20]:
# Save the model
recognition_model.save('license_plate_recognition_model.h5')
print("Model saved as 'license_plate_recognition_model.h5'.")

Model saved as 'license_plate_recognition_model.h5'.


In [23]:
# Load the saved model
model = tf.keras.models.load_model('license_plate_recognition_model.h5')

# Function to recognize license plate text
def recognize_license_plate(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 64))  # Resize to match model input
    image = image / 255.0  # Normalize

    # Predict the text
    predicted_label = model.predict(np.expand_dims(image, axis=0))
    predicted_text = label_encoder.inverse_transform([np.argmax(predicted_label)])[0]
    return predicted_text

# Test on a new image
new_image_path = r'C:\Users\user\Desktop\licplt new.jpeg'
predicted_text = recognize_license_plate(new_image_path)
print(f"Predicted License Plate Text: {predicted_text}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
Predicted License Plate Text: 7355-55906


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode text labels into numerical labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(text_labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cropped_images, encoded_labels, test_size=0.2, random_state=42)

# Define the CNN model
def create_recognition_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the model
input_shape = (64, 128, 3)  # Adjust based on your image size
num_classes = len(label_encoder.classes_)
recognition_model = create_recognition_model(input_shape, num_classes)
recognition_model.summary()

# Train the model
recognition_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

NameError: name 'text_labels' is not defined

In [4]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Step 1: Load the annotations
annotations = pd.read_csv('automatic_annotations.csv')

# Step 2: Preprocess the data
car_images_path = r'C:\Users\user\Downloads\DATA SCIENTIST_ASSIGNMENT-20250226T162947Z-001\DATA SCIENTIST_ASSIGNMENT\test\test\test'
output_folder = 'cropped_license_plates'
os.makedirs(output_folder, exist_ok=True)

cropped_images = []
text_labels = []

for index, row in annotations.iterrows():
    filename = row['filename']
    xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
    text = row['text']

    # Load the image
    image_path = os.path.join(car_images_path, filename)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        continue

    # Validate bounding box coordinates
    height, width, _ = image.shape
    if xmin >= xmax or ymin >= ymax:
        print(f"Invalid bounding box for {filename}: ({xmin}, {ymin}, {xmax}, {ymax})")
        continue
    if xmin < 0 or ymin < 0 or xmax > width or ymax > height:
        print(f"Bounding box out of bounds for {filename}: ({xmin}, {ymin}, {xmax}, {ymax})")
        continue

    # Crop the license plate
    license_plate = image[ymin:ymax, xmin:xmax]
    if license_plate.size == 0:
        print(f"Empty license plate for {filename}: ({xmin}, {ymin}, {xmax}, {ymax})")
        continue

    # Resize and normalize the license plate
    license_plate = cv2.resize(license_plate, (128, 64))  # Resize to match model input
    license_plate = license_plate / 255.0  # Normalize to [0, 1]

    # Save the cropped license plate
    output_path = os.path.join(output_folder, f"cropped_{filename}")
    cv2.imwrite(output_path, license_plate * 255)  # Save as 8-bit image

    # Append to lists
    cropped_images.append(license_plate)
    text_labels.append(text)

# Convert to numpy arrays
cropped_images = np.array(cropped_images)
text_labels = np.array(text_labels)

print(f"Preprocessed {len(cropped_images)} license plates.")

# Step 3: Encode text labels into numerical labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(text_labels)

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cropped_images, encoded_labels, test_size=0.2, random_state=42)

# Step 5: Define the CNN model
def create_recognition_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Step 6: Create and train the model
input_shape = (64, 128, 3)  # Adjust based on your image size
num_classes = len(label_encoder.classes_)
recognition_model = create_recognition_model(input_shape, num_classes)
recognition_model.summary()

# Train the model
recognition_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 7: Save the model
recognition_model.save('license_plate_recognition_model.h5')
print("Model saved as 'license_plate_recognition_model.h5'.")

Invalid bounding box for 1066.jpg: (218, 449, 326, 449)
Bounding box out of bounds for 1092.jpg: (766, 7, 855, 45)
Invalid bounding box for 907.jpg: (185, 198, 312, 198)
Preprocessed 402 license plates.


C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 126, 32)         │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 26880)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,440,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 375)                 │          48,375 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,508,535 (13.38 MB)

 Trainable params: 3,508,535 (13.38 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 178ms/step - accuracy: 0.0000e+00 - loss: 6.1236 - val_accuracy: 0.0000e+00 - val_loss: 5.9392
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.0055 - loss: 5.9237 - val_accuracy: 0.0000e+00 - val_loss: 5.9590
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 139ms/step - accuracy: 0.0000e+00 - loss: 5.9093 - val_accuracy: 0.0000e+00 - val_loss: 6.0203
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.0000e+00 - loss: 5.8758 - val_accuracy: 0.0123 - val_loss: 6.0908
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.0138 - loss: 5.7985 - val_accuracy: 0.0247 - val_loss: 6.2238
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - accuracy: 0.0288 - loss: 5.6963 - val_accuracy: 0.0123 - val_loss: 6.7829
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.0269 - loss: 5.5603 - val_accuracy: 0.0123 - val_loss: 6.9620
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.0680 - loss: 5

Model saved as 'license_plate_recognition_model.h5'.


In [5]:
# Load the saved model
model = tf.keras.models.load_model('license_plate_recognition_model.h5')

# Function to recognize license plate text
def recognize_license_plate(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return None

    # Resize and normalize the image
    image = cv2.resize(image, (128, 64))  # Resize to match model input
    image = image / 255.0  # Normalize

    # Predict the text
    predicted_label = model.predict(np.expand_dims(image, axis=0))
    predicted_text = label_encoder.inverse_transform([np.argmax(predicted_label)])[0]
    return predicted_text

# Test on a new image
new_image_path = r'C:\Users\user\Desktop\licplt new.jpeg'
predicted_text = recognize_license_plate(new_image_path)
if predicted_text is not None:
    print(f"Predicted License Plate Text: {predicted_text}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
Predicted License Plate Text: 1733 59116


In [10]:
# Load the saved model
model = tf.keras.models.load_model('license_plate_recognition_model.h5')

# Function to recognize license plate text
def recognize_license_plate(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return None

    # Resize and normalize the image
    image = cv2.resize(image, (128, 64))  # Resize to match model input
    image = image / 255.0  # Normalize

    # Predict the text
    predicted_label = model.predict(np.expand_dims(image, axis=0))
    predicted_text = label_encoder.inverse_transform([np.argmax(predicted_label)])[0]

    # Post-process the predicted text
    predicted_text = predicted_text.replace(" ", "")  # Remove spaces
    return predicted_text

# Test on a new image
new_image_path = r'C:\Users\user\Desktop\licplt2.jpg'
predicted_text = recognize_license_plate(new_image_path)
if predicted_text is not None:
    print(f"Predicted License Plate Text: {predicted_text}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Predicted License Plate Text: [122_;2171]


In [11]:
# Save the cropped license plate for inspection
cv2.imwrite('cropped_license_plate.jpg', image * 255)

True

In [12]:
print("Label Encoder Classes:", label_encoder.classes_)

Label Encoder Classes: ['&*5' '&0)Lec' "'14264" "'160 0*; 8806|" "'195" "'2580" "'354820" "'3989"
 "'79 3 8612" "'9" "'MO" '(165' '(47' '(81 5i' '(9L 35 13651' '(ET' '(j'
 '010o' '01263*' '01650-7878' '056581]' '0n13-52788]' '0ny 1823' '100'
 '100+ 5 5333' '1001714' '1018"; 6950 ' '10335 9640' '104 3' '105 * L1'
 '105 3*y 9699' '108' '10g9 35 4225' '1103*53517' '110343 6829}'
 '1116*53668]' '112' '112 + 5633' '113s Z39L' '116 347 3C68' '1163*'
 '1187' '1190*5 6926' '119032123' '11L O5 239' '120> 89' '121' '123'
 '125 053 6088 ' '126' '129' '13' '130 2*3 9996' '131' '131* ; 870'
 '1316 5 40L5]' '132854' '135 07; 4716' '135 _; 5906' '135*4 5332'
 '13735 8 159' '138 363 1769' '1396' '13oo' '140' '140 3*9 659'
 '140081714|' '141.356603' '1433' '144_ ' '145 333 1093' '146v2'
 '1488 52903' '15029 |' '151' '151231866' '152  3567' '152 _; 9438' '1526'
 '1530*58959' '1558' '156 _' '159' '15}' '160 3*5 5065' '160*794041'
 '1608754041"' '1612' '1618-33230' '16264' '164' '164 349' '167* 2142'
 '16